# Getting started with Dask on Saturn Cloud

Dask is a framework that easily lets you run Python in parallel across distributed machines. Below is an example of taking a function that computes exponents and running it across a list of inputs in parallel

This code imports the Dask libraries and connects to the Saturn Cloud dask cluster. If you are having issues with this, you may not have a running dask cluster, you can adjust the dask cluster on the project page.

In [13]:
import dask
from dask_saturn import SaturnCluster
from dask.distributed import Client

cluster = SaturnCluster()
client = Client(cluster)

[2021-02-19 23:03:10] INFO - dask-saturn | Cluster is ready
[2021-02-19 23:03:10] INFO - dask-saturn | Registering default plugins
[2021-02-19 23:03:11] INFO - dask-saturn | {'tcp://10.0.26.85:42489': {'status': 'repeat'}}


The `@dask.delayed` decorator indicates this will run lazily in parallel:

In [14]:
@dask.delayed
def lazy_exponent(args):
    x,y = args
    '''Define a lazily evaluating function'''
    return x ** y

With this dask function, we can now run the function across a distributed set of workers using the `client` functions. In this case we take thios list of pairs of numbers and run them in a distributed setting. After using `client.compute()` to actually compute the values (since they're otherwise lazy) we can then look at the results by using `x.result()` for each element of the computed list:

In [14]:
inputs = [[1,2], [3,4], [5,6], [9, 10], [11, 12]]

example_future = client.map(lazy_exponent, inputs)
futures_gathered = client.gather(example_future)
futures_computed = client.compute(futures_gathered, sync=False)

results = [x.result() for x in futures_computed]
results

[1, 81, 15625, 3486784401, 3138428376721]

This was _somewhat_ of a toy example--you probably want to do more complex computations than exponents with dask. However the core concept of making a function and then running it in a distributed fashion is at the core of what you can do with Dask on Saturn Cloud.


When you're done, you can close the connection to the cluster:

In [ ]:
client.close()